### 导包

In [20]:
import os
os.getcwd()
import sys
print(sys.version)




3.11.4 (v3.11.4:d2340ef257, Jun  6 2023, 19:15:51) [Clang 13.0.0 (clang-1300.0.29.30)]


In [22]:
pip install SQLAlchemy-2.0.25-py3-none-any.whl


Processing ./SQLAlchemy-2.0.25-py3-none-any.whl

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
#pip install pip-23.3.2-py3-none-any.whl
pip install xgboost

SyntaxError: invalid syntax (1675373398.py, line 2)

In [40]:
pip install xgboost-2.0.3-py3-none-macosx_12_0_arm64.whl

Processing ./xgboost-2.0.3-py3-none-macosx_12_0_arm64.whl
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install lightgbm-4.2.0.tar.gz

Processing ./lightgbm-4.2.0.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 2
  ╰─> [517 lines of output]
        Using cached pyproject_metadata-0.7.1-py3-none-any.whl (7.4 kB)
        Using cached ninja-1.11.1.1-py2.py3-none-macosx_10_9_universal2.macosx_10_9_x86_64.macosx_11_0_arm64.macosx_11_0_universal2.whl.metadata (5.3 kB)
        Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/270.6 kB ? eta -:--:--
         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/270.6 kB ? eta -:--:--
         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/270.6 kB ? eta -:--:--
         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/270.6 kB ? eta -:--:--
         ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/270.6 kB ? 

In [50]:
pip install 'lightgbm[scikit-learn]'

ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/92/3d/209b257b7bcfaa39d54b9ab22db5097ff110c6a2f2399d5244c76c43aba2/lightgbm-4.2.0.tar.gz (Caused by ConnectTimeoutError(<pip._vendor.urllib3.connection.HTTPSConnection object at 0x10560d5d0>, 'Connection to files.pythonhosted.org timed out. (connect timeout=15)'))

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install toad-0.1.3-cp311-cp311-macosx_10_9_universal2.whl


Processing ./toad-0.1.3-cp311-cp311-macosx_10_9_universal2.whl
Note: you may need to restart the kernel to use updated packages.


In [6]:
import re
import os
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import warnings
import toad
warnings.filterwarnings('ignore')
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import roc_curve, auc
import xgboost as xgb
from xgboost.sklearn import XGBClassifier,XGBRegressor
#import lightgbm as lgb
import matplotlib.pyplot as plt
import gc
import json

### 原数据读取

#### 原始数据

In [7]:
def data_pred():
    df_init=pd.read_csv('accepted_2007_to_2018Q4.csv')
    df=df_init.sample(50000).reset_index(drop=True)
    
    df['y']=df.loan_status.map({'Fully Paid':0,'Late (31-120 days)':1,'Late (16-30 days)':1})
    df_copy=df[df.y.isin([0,1])].copy()
    return df_copy.reset_index(drop=True)

df=data_pred()
df.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,y
0,99857,77576694,NaN,5300.0,5300.0,5300.0,36 months,25.11,211.04,F,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,37399,102466807,NaN,21000.0,21000.0,21000.0,60 months,12.74,475.03,C,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,46111,134888740,NaN,12000.0,12000.0,12000.0,36 months,10.07,387.61,B,...,NaN,DirectPay,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,7489,59292907,NaN,23150.0,23150.0,23150.0,60 months,11.53,509.48,B,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,98947,63489038,NaN,25000.0,25000.0,24900.0,60 months,12.69,564.87,C,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [8]:
df.select_dtypes(include=['string','object']).head()

,id,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,...,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
0,77576694,36 months,F,F1,NaN,NaN,RENT,Verified,May-2016,Fully Paid,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
1,102466807,60 months,C,C1,Co-Owner - COO,9 years,RENT,Not Verified,Mar-2017,Fully Paid,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
2,134888740,36 months,B,B2,Tech,10+ years,OWN,Source Verified,Jun-2018,Fully Paid,...,NaN,NaN,NaN,NaN,NaN,DirectPay,N,NaN,NaN,NaN
3,59292907,60 months,B,B5,NaN,NaN,MORTGAGE,Not Verified,Sep-2015,Fully Paid,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
4,63489038,60 months,C,C2,Dir of Supply Chain,2 years,MORTGAGE,Not Verified,Nov-2015,Fully Paid,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN


In [9]:
date_fea=[
    'issue_d',
    'earliest_cr_line',
    'last_pymnt_d',
    'last_credit_pull_d',
    'sec_app_earliest_cr_line',
    'hardship_start_date',
    'hardship_end_date',
    'payment_plan_start_date',
    'debt_settlement_flag_date',
    'settlement_date',
    'next_pymnt_d'
]

target_label_fea=[
    'sub_grade',
    'emp_title',
    'home_ownership',
    'verification_status',
    'pymnt_plan',
    'purpose',
    'addr_state',
    'initial_list_status',
    'application_type',
    'verification_status_joint',
    'hardship_flag',
    'hardship_type',
    'hardship_reason',
    'hardship_status',
    'hardship_loan_status',
    'disbursement_method',
    'debt_settlement_flag',
    'settlement_status',        
]

In [19]:
def target_label_mean(df_train,col,y_name='y'): # 单变量目标编码
    return dict(round(df_train.groupby([col])[y_name].mean(),6))  

def target_label_mean_all(df_train,col_list,y_name='y'): # 全部变量目标编码
    return {col:target_label_mean(df_train,col,y_name) for col in col_list}

def train_test_split_self(df_copy,y_name='y'):  # 数据划分
    frac=0.7
    train_index=list(df_copy[df_copy[y_name]==1].sample(frac=frac,random_state=1).index)+list(df_copy[df_copy[y_name]==0].sample(frac=frac,random_state=1).index)
    df_copy.loc[df_copy.index.isin(train_index),'sample_label']='train'
    df_copy.loc[~df_copy.index.isin(train_index),'sample_label']='test'
    
    return df_copy

def train_test_target_label(df_copy,col_list):  # df目标编码
    df=df_copy.pipe(train_test_split_self)
    all_col_map=target_label_mean_all(df[df.sample_label=='train'],col_list,y_name='y')
    for col in col_list:
        df[col]=df[col].map(all_col_map[col])
    
    return df,all_col_map

#def get_fill_df(df_pred,y_name='y'):  # 缺失填充
    #df=df_pred.copy()
    #fill_col=[col for col in df.columns if col not in [y_name]]
    #df=df.fillna(df.mean())

    #return df


    
def get_fill_df(df_pred, y_name='y'):
    df = df_pred.copy()
    
    # 选择数值类型的列
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    # 计算数值类型列的平均值
    mean_values = df[numeric_cols].mean()

    # 只填充数值类型列的缺失值
    df[numeric_cols] = df[numeric_cols].fillna(mean_values)

    return df

 
def fea_engineer(df_copy):  # 所有预处理汇总
    df=df_copy.copy()
    df.term=df.term.str.replace('months','').str.replace(' ','').astype('float')
    
    grade_map=dict(enumerate(list('ABCDEFG')))
    grade_map={value:key for key,value in grade_map.items()}
    df.grade=df.grade.map(grade_map)
    
    df.emp_length=df.emp_length.str.replace('years','').str.replace('year','').str.replace(' ','').map({'<1':0,'10+':10})
    
    float_col=list(df_copy.select_dtypes(include=['int','float']).columns)
    float_col.remove('y')
    
    drop_col=['member_id','loan_status','url','desc','title','zip_code']
    df=df.drop(drop_col+date_fea,axis=1)  # 日期字段含义不明确，所以先不用了
    
    
    df,all_col_map=train_test_target_label(df,target_label_fea)  # 部分类别型变量进行target_label编码
    return df,all_col_map

df_pred,all_col_map=fea_engineer(df)
df_pred.head()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,settlement_status,settlement_amount,settlement_percentage,settlement_term,y,sample_label
0,99857,77576694,5300.0,5300.0,5300.0,36.0,25.11,211.04,5,0.030928,...,NaN,NaN,0.024161,0.024002,NaN,NaN,NaN,NaN,0.0,train
1,37399,102466807,21000.0,21000.0,21000.0,60.0,12.74,475.03,2,0.028803,...,NaN,NaN,0.024161,0.024002,NaN,NaN,NaN,NaN,0.0,test
2,46111,134888740,12000.0,12000.0,12000.0,36.0,10.07,387.61,1,0.012161,...,NaN,NaN,0.164706,0.024002,NaN,NaN,NaN,NaN,0.0,train
3,7489,59292907,23150.0,23150.0,23150.0,60.0,11.53,509.48,1,0.024482,...,NaN,NaN,0.024161,0.024002,NaN,NaN,NaN,NaN,0.0,train
4,98947,63489038,25000.0,25000.0,24900.0,60.0,12.69,564.87,2,0.026210,...,NaN,NaN,0.024161,0.024002,NaN,NaN,NaN,NaN,0.0,train


In [20]:
df_pred_fill=get_fill_df(df_pred,y_name='y')
df_pred_fill

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,settlement_status,settlement_amount,settlement_percentage,settlement_term,y,sample_label
0,99857,77576694,5300.0,5300.0,5300.0,36.0,25.11,211.04,5,0.030928,...,11094.541809,199.509468,0.024161,0.024002,1.0,5111.578947,51.052632,14.789474,0.0,train
1,37399,102466807,21000.0,21000.0,21000.0,60.0,12.74,475.03,2,0.028803,...,11094.541809,199.509468,0.024161,0.024002,1.0,5111.578947,51.052632,14.789474,0.0,test
2,46111,134888740,12000.0,12000.0,12000.0,36.0,10.07,387.61,1,0.012161,...,11094.541809,199.509468,0.164706,0.024002,1.0,5111.578947,51.052632,14.789474,0.0,train
3,7489,59292907,23150.0,23150.0,23150.0,60.0,11.53,509.48,1,0.024482,...,11094.541809,199.509468,0.024161,0.024002,1.0,5111.578947,51.052632,14.789474,0.0,train
4,98947,63489038,25000.0,25000.0,24900.0,60.0,12.69,564.87,2,0.026210,...,11094.541809,199.509468,0.024161,0.024002,1.0,5111.578947,51.052632,14.789474,0.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24180,11586,71613139,35000.0,35000.0,35000.0,60.0,21.18,950.42,4,0.073034,...,11094.541809,199.509468,0.024161,0.024002,1.0,5111.578947,51.052632,14.789474,0.0,train
24181,36061,84667040,7500.0,7500.0,7500.0,36.0,12.79,251.95,2,0.028803,...,11094.541809,199.509468,0.024161,0.024002,1.0,5111.578947,51.052632,14.789474,0.0,test
24182,55620,114024518,5000.0,5000.0,5000.0,36.0,15.05,173.45,2,0.035595,...,11094.541809,199.509468,0.024161,0.024002,1.0,5111.578947,51.052632,14.789474,0.0,train
24183,56015,26259442,8000.0,8000.0,8000.0,36.0,20.20,298.13,4,0.073034,...,11094.541809,199.509468,0.024161,0.024002,1.0,5111.578947,51.052632,14.789474,0.0,test


In [21]:
df.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,y
0,99857,77576694,NaN,5300.0,5300.0,5300.0,36 months,25.11,211.04,F,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,37399,102466807,NaN,21000.0,21000.0,21000.0,60 months,12.74,475.03,C,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,46111,134888740,NaN,12000.0,12000.0,12000.0,36 months,10.07,387.61,B,...,NaN,DirectPay,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,7489,59292907,NaN,23150.0,23150.0,23150.0,60 months,11.53,509.48,B,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,98947,63489038,NaN,25000.0,25000.0,24900.0,60 months,12.69,564.87,C,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [22]:
fea_list=list(df_pred.drop(['id','y','sample_label'],axis=1).columns)
len(fea_list)

134

### 变量统计

#### iv_miss

In [23]:
import toad
def iv_miss(df,var_list,y):
    df_tmp=df[df[y].notnull()].copy()
    
    iv_all=toad.quality(df_tmp[var_list+[y]], target=y, indicators = ['iv','unique'])[['unique','iv']]

    miss_per=pd.DataFrame(df[var_list].isnull().sum()/(df.shape[0]))
    miss_per.columns=['缺失率']
    result=pd.concat([miss_per,iv_all],axis=1)
    return result.sort_values('iv',ascending=False)


In [24]:
df_iv=iv_miss(df_pred,fea_list,'y') # iv计算
df_iv

,缺失率,unique,iv
out_prncp_inv,0.000000,602.0,16.455845
out_prncp,0.000000,602.0,16.455845
last_pymnt_amnt,0.000000,22756.0,3.555538
emp_title,0.209965,53.0,3.386996
total_rec_prncp,0.000000,1757.0,2.141958
...,...,...,...
verification_status,0.000000,3.0,0.006831
delinq_amnt,0.000000,82.0,0.001684
policy_code,0.000000,1.0,0.000000
collection_recovery_fee,0.000000,1.0,0.000000


In [25]:
y_name='y'
# 变量筛选
train_selected,drop_col=toad.selection.select(df_pred[fea_list+[y_name]],y_name,empty=0.9,
                                     iv=0.02,corr=0.7,return_drop=True)
train_selected

,Unnamed: 0,term,int_rate,emp_title,home_ownership,annual_inc,purpose,addr_state,dti,fico_range_low,...,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,pct_tl_nvr_dlq,pub_rec_bankruptcies,hardship_flag,disbursement_method,debt_settlement_flag,y
0,99857,36.0,25.11,NaN,0.028737,15636.0,0.027403,0.025510,28.40,685.0,...,17.0,8.0,12.0,0.0,100.0,1.0,0.024291,0.024161,0.024002,0.0
1,37399,60.0,12.74,NaN,0.028737,72000.0,0.027403,0.028037,26.57,740.0,...,10.0,5.0,12.0,0.0,100.0,0.0,0.024291,0.024161,0.024002,0.0
2,46111,36.0,10.07,0.111111,0.028322,60000.0,0.016233,0.034130,18.66,715.0,...,3.0,1.0,6.0,0.0,100.0,0.0,0.024291,0.164706,0.024002,0.0
3,7489,60.0,11.53,NaN,0.021115,83000.0,0.016233,0.004950,29.00,710.0,...,25.0,8.0,17.0,0.0,98.3,0.0,0.024291,0.024161,0.024002,0.0
4,98947,60.0,12.69,0.000000,0.021115,125000.0,0.027403,0.020446,37.84,710.0,...,45.0,8.0,23.0,0.0,100.0,0.0,0.024291,0.024161,0.024002,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24180,11586,60.0,21.18,0.000000,0.021115,146000.0,0.024129,0.020446,21.94,660.0,...,17.0,9.0,20.0,0.0,94.4,0.0,0.024291,0.024161,0.024002,0.0
24181,36061,36.0,12.79,NaN,0.028737,26000.0,0.027403,0.024527,19.58,675.0,...,8.0,2.0,6.0,0.0,90.0,0.0,0.024291,0.024161,0.024002,0.0
24182,55620,36.0,15.05,0.000000,0.021115,52176.0,0.016233,0.024527,3.63,675.0,...,10.0,5.0,7.0,0.0,100.0,0.0,0.024291,0.024161,0.024002,0.0
24183,56015,36.0,20.20,0.000000,0.028737,103000.0,0.056075,0.025510,12.80,660.0,...,34.0,15.0,19.0,0.0,94.4,1.0,0.024291,0.024161,0.024002,0.0


In [26]:
# 剩余变量列表
fea_list_select=list(train_selected.drop('y',axis=1).columns)
print(len(fea_list),len(fea_list_select))

134 60


### 建模

#### 建模函数

In [13]:
def init_params(model_select='lgb'):  # 初始化参数
    params_lr={
        'n_jobs':4,
        'max_iter':100,
        'penalty':'l2',
        'solver':'lbfgs',
        'random_state':1,
        'tol':0.0001,
        'C':1,
        'verbose':1,
    }
    params_rf={
        'n_jobs':4,
        'max_samples':0.8,
        'n_estimators':500,
        'max_depth':3,
        'min_samples_leaf':50,
        'max_features':'auto',
        'min_impurity_decrease':0,
        'bootstrap':True,
        'oob_score':True,
        'random_state':1,
        'verbose':1
    }
    params_xgb={
        'objective':'binary:logistic',
        'eval_metric':'auc',
        'nthread':4,
        'n_estimators':500,
        'eta':0.02,
        'max_depth':3,
        'min_child_weight':50,
        'scale_pos_weight':1,
        'gamma':10,
        'reg_alpha':2,
        'reg_lambda':2,
        'subsample':0.8,
        'colsample_bytree':0.8,
        'seed':123
    }
    params_lgb={
        'boosting_type': 'gbdt', 
        'objective': 'binary',
        'metric':'auc',
        'n_jobs': 4,
        'n_estimators':params_xgb.get('n_estimators',100),
        'learning_rate': params_xgb.get('eta',0.1), 
        'max_depth':params_xgb.get('max_depth',4), 
        'num_leaves': params_xgb.get('num_leaves',20), 
        'max_bin':255,
        'subsample_for_bin':100000, # 构建直方图的样本量
        'min_split_gain':params_xgb.get('gamma',10),
        'min_child_samples':params_xgb.get('min_child_weight',50),
        'colsample_bytree': params_xgb.get('colsample_bytree',0.8),
        'subsample': params_xgb.get('subsample',0.8), 
        'subsample_freq': 1,   # 每 k 次迭代执行bagging      
        'feature_fraction_seed':2,
        'bagging_seed': 1,
        'reg_alpha':params_xgb.get('reg_alpha',10),
        'reg_lambda':params_xgb.get('reg_lambda',10),
        'scale_pos_weight':params_xgb.get('scale_pos_weight',1), # 等价于is_unbalance=False
        'silent':True,
        'random_state':1,
        'verbose':-1, # 控制模型训练过程的输出信息，-1为不输出信息
     }
    if model_select=='lr':
        return params_lr
    if model_select=='rf':
        return params_rf
    if model_select=='xgb':
        return params_xgb
    if model_select=='lgb':
        return params_lgb

def ks_auc_value(y_value,df,model): # 计算ks、auc
    y_pred=model.predict_proba(df)[:,1]
    fpr,tpr,thresholds= roc_curve(list(y_value),list(y_pred))
    ks=max(tpr-fpr)
    auc= roc_auc_score(list(y_value),list(y_pred))
    return ks,auc

def model_train_sklearn(df,y_name,model_fea,model_select='lgb'):
    
    params=init_params(model_select=model_select)
    if model_select=='xgb':
        model=XGBClassifier(**params)
    elif model_select=='lgb':
        model=lgb.LGBMClassifier(**params)
    elif model_select=='rf':
        model=RandomForestClassifier(**params)
    elif model_select=='lr':
        model=LogisticRegression(**params)
        
    x_train,y_train=df[df.sample_label=='train'][model_fea],df[df.sample_label=='train'][y_name]
    x_test,y_test=df[df.sample_label=='test'][model_fea],df[df.sample_label=='test'][y_name]
    
    if model_select in ['rf','lr']:
        model.fit(x_train,y_train)
    else:
        model.fit(x_train,y_train,eval_set=[(x_train, y_train),(x_test, y_test)],verbose=True)
    
    train_ks,train_auc=ks_auc_value(y_train,x_train,model)
    test_ks,test_auc=ks_auc_value(y_test,x_test,model)
    
    model_result={
        'train_good':(y_train.count()-y_train.sum()),'train_bad':y_train.sum(),
        'test_good':(y_test.count()-y_test.sum()),'test_bad':y_test.sum(),
        'train_ks':train_ks,'train_auc':train_auc,
        'test_ks':test_ks,'test_auc':test_auc,
    }
    return model_result,model


In [ ]:
xgb_model_result,xgb_model=model_train_sklearn(df_pred,'y',fea_list_select,model_select='xgb')
lgb_model_result,lgb_model=model_train_sklearn(df_pred,'y',fea_list_select,model_select='lgb')
rf_model_result,rf_model=model_train_sklearn(df_pred_fill,'y',fea_list_select,model_select='rf')
lr_model_result,lr_model=model_train_sklearn(df_pred_fill,'y',fea_list_select,model_select='lr')

In [ ]:
pd.DataFrame([xgb_model_result,lgb_model_result,rf_model_result,lr_model_result],index=['xgb','lgb','rf','lr'])